In [86]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

In [328]:
def field_selection(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # converted to hsv
    image_shape = img.shape
    img_area = int(image_shape[0] * image_shape[1])
    lower_green = np.array([15, 50, 50])  # lower bound of green
    upper_green = np.array([70, 255, 255])  # upper bound of green
    mask = cv2.inRange(hsv, lower_green, upper_green)  # creates a mask where pixels within the range are white and others are black
    mask = cv2.GaussianBlur(mask, (7, 7), cv2.BORDER_DEFAULT)  # applies a blur

    (contours, _) = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # finds contours, only external ones and only the endpoints

    # check if there are any contours
    if contours:
    # keep track of largest and second largest contours and areas
        max_area = 1
        scd_max_area = 0
        max_index = 0
        scd_max_index = 0

        # find area of contours and keep track of largest and second largest
        for i, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if area > max_area:
                scd_max_area = max_area
                scd_max_index = max_index
                max_area = area
                max_index = i
            elif area > scd_max_area:
                scd_max_area = area
                scd_max_index = i
        # selects largest contour and computes the convex hull to create a convex polygon
        cnt = contours[max_index]
        hull = cv2.convexHull(cnt)
    else:
        field_in = np.zeros_like(img)
        return field_in

    field_mask = np.zeros((image_shape[0], image_shape[1]))

    # checks if the largest contour is at least 50% of the image, draws it if so
    if max_area > 0.5 * img_area:
        cv2.drawContours(field_mask, [hull], 0, 255, -1)  # draws convex hull on mask, filling it with white (255)
        # checks if second largest contour is at least 10% of the largest contour, draws it if so
        if scd_max_area > 0.2 * max_area:
            cnt = contours[scd_max_index]
            hull = cv2.convexHull(cnt)
            cv2.drawContours(field_mask, [hull], 0, 255, -1)
    else:
        contour_points = np.array([[0, 0], [0, image_shape[0]], [image_shape[1], image_shape[0]], [image_shape[1], 0]])
        cv2.drawContours(field_mask, [contour_points],  0, 255, -1)

    img2 = img.copy()
    cv2.drawContours(img2, [cv2.convexHull(contours[max_index])], -1, (255, 0, 0), 3)
    cv2.drawContours(img2, [cv2.convexHull(contours[scd_max_index])], -1, (0, 255, 0), 3)
    # cv2.drawContours(img2, [contours[max_index]], -1, (0, 0, 255), 3)
    # cv2.drawContours(img2, [contours[scd_max_index]], -1, (0, 0, 0), 3)

    # copies pixels from original image where mask is white
    field_in = np.zeros_like(img)
    field_in[field_mask == 255, :] = img[field_mask == 255, :]

    # same thing but where mask is black
    field_out = np.zeros_like(img) * 255
    field_out[field_mask == 0, :] = img[field_mask == 0, :]
    return field_in, field_mask, img2

In [ ]:
x = str(24).zfill(5)
rand = str(np.random.randint(0, len(os.listdir('frames')))).zfill(5)

img = cv2.imread(f'frames/{rand}.jpg')
field_in = field_selection(img)[0]
field_mask = field_selection(img)[1]
contours = field_selection(img)[2]

fig, ax = plt.subplots(2, 2, figsize=(16, 9))
ax[0, 0].imshow(img); ax[0, 0].axis('off'); ax[0, 0].set_title(rand)
ax[0, 1].imshow(contours); ax[0, 1].axis('off'); ax[0, 1].set_title('Field')
ax[1, 0].imshow(field_mask); ax[1, 0].axis('off'); ax[1, 0].set_title('Mask')
ax[1, 1].imshow(field_in); ax[1, 1].axis('off'); ax[1, 1].set_title('Field')

plt.show()

In [315]:
for filename in os.listdir('frames'):
    if filename.endswith('.jpg'):
        input_image_path = os.path.join('frames', filename)
        img = cv2.imread(input_image_path)
        field_in = field_selection(img)[0]

        output_image_path = os.path.join('field_frames', filename)
        cv2.imwrite(output_image_path, field_in)